In [ ]:
# define templates
from datetime import datetime
from pydantic import BaseModel, Field
from typing import Optional

class LineItem(BaseModel):
    """A line item in an invoice."""
    item_id: str = Field(description = "Item identifier")
    item_name: str = Field(description="The name of this item")
    price: float = Field(description="The price of this item")


class Invoice(BaseModel):
    """A representation of information from an invoice."""

    invoice_id: str = Field(
        description="A unique identifier for this invoice, often a number"
    )
    date: datetime = Field(description="The date this invoice was created")
    line_items: list[LineItem] = Field(
        description="A list of all the items in this invoice"
    )
    quantity: float = Field(description="The bought quantity")
    vat:str = Field(description="The vat percentage applied, if applied. Otherwise, N/A.")
    gross_amount: float = Field(description="Invoice Gross amount.")

class BinaryAnswer(BaseModel):
    """A structured binary answer to a question."""

    answer: bool = Field(description="The answer to the question. True for yes, False for no. None otherwise.")
    details: Optional[str] = Field(description="Explanation related to the answer.")

In [ ]:
# define tools
from llama_index.core.program.function_program import get_function_tool

invoice_tool = get_function_tool(Invoice)
binary_answer_tool = get_function_tool(BinaryAnswer)

In [ ]:
# define agent
from llama_index.core.agent.workflow import AgentWorkflow, FunctionAgent
from llama_index.core.tools import FunctionTool

In [ ]:
# define llm
from llama_index.llms.ollama import Ollama

phi_llm = Ollama(
    model="phi4-mini:latest",
    base_url="http://localhost:11434", 
    request_timeout=360.0,
    temperature=0.01
    )

minion_llm = Ollama(
    model="gemma3:4b",
    base_url="http://localhost:11434", 
    request_timeout=360.0,
    temperature=0.01
    )

#response = llm.complete("What is the capital of France?")
#print(response)

In [ ]:
# Create specialized agents
extract_agent = FunctionAgent(
    name="ExtractAgent",
    llm=phi_llm,
    description="Extract structured information from a provided based on given tools. Use always tool.",
    system_prompt="You are a meticoulous document reviewer able to extract specific information...",
    tools=[invoice_tool],
    can_handoff_to=["AnswerAgent"]
)

answer_agent = FunctionAgent(
    name = "AnswerAgent",
    llm=phi_llm,
    description="Provide the final answer with the most suitable structure based on available tools. Use always tool.",
    system_prompt="You are precise answerer...",
    tools=[binary_answer_tool]
)

# Create the workflow
agent_workflow = AgentWorkflow(
    agents=[extract_agent, answer_agent],
    
    root_agent="ExtractAgent"
)

In [ ]:
response = await agent_workflow.run(user_msg="What is the weather in San Francisco?")

In [ ]:
response

In [ ]:
from pathlib import Path

all_files_gen = Path("./data/").rglob("*")
all_files = [f.resolve() for f in all_files_gen]
all_pdf_files = [f for f in all_files if f.suffix.lower() == ".pdf"]
len(all_pdf_files)

In [ ]:
from llama_index.core import Document
from docling.document_converter import DocumentConverter

converter = DocumentConverter()

doc_limit = 100

docs = []

loaded_docs = converter.convert_all(all_pdf_files[:])

text=converter.convert(all_pdf_files[-1]).document.export_to_markdown()


In [ ]:
prompt = f"The gross amount is higher than 60 euros?This is the document:{text}. Provide a structured answer."

In [ ]:
response = await agent_workflow.run(user_msg=prompt)
print(str(response))

In [ ]:
handler = agent_workflow.run(user_msg=prompt)


In [ ]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

current_agent = None
current_tool_calls = ""
async for event in handler.stream_events():
    if (
        hasattr(event, "current_agent_name")
        and event.current_agent_name != current_agent
    ):
        current_agent = event.current_agent_name
        print(f"\n{'='*50}")
        print(f"🤖 Agent: {current_agent}")
        print(f"{'='*50}\n")

    # if isinstance(event, AgentStream):
    #     if event.delta:
    #         print(event.delta, end="", flush=True)
    # elif isinstance(event, AgentInput):
    #     print("📥 Input:", event.input)
    elif isinstance(event, AgentOutput):
        if event.response.content:
            print("📤 Output:", event.response.content)
        if event.tool_calls:
            print(
                "🛠️  Planning to use tools:",
                [call.tool_name for call in event.tool_calls],
            )
    elif isinstance(event, ToolCallResult):
        print(f"🔧 Tool Result ({event.tool_name}):")
        print(f"  Arguments: {event.tool_kwargs}")
        print(f"  Output: {event.tool_output}")
    elif isinstance(event, ToolCall):
        print(f"🔨 Calling Tool: {event.tool_name}")
        print(f"  With arguments: {event.tool_kwargs}")

In [ ]:
sllm = llm.as_structured_llm(Invoice)

In [ ]:
handler = sllm.complete(prompt=prompt)

In [ ]:
structured_info = handler.text
from llama_index.core.prompts import PromptTemplate


answer_prompt = PromptTemplate('Given an answer to the following question based on the given context.\nQuestion:{question}\nContext:{context}')
llm.structured_predict(BinaryAnswer, answer_prompt, question="L'importo totale è minore di 50 euro?", context=structured_info)

In [ ]:
structured_info